In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

부스팅(Boosting)  
초기에는 모든 데이터 포인트에 동일한 가중치를 할당하고 점차 학습이 진행되면서 올바르게 분류된 데이터 포인트의 가중치는 감소시키는 반면에 잘못 분류된 데이터 포인트의 가중치는 증가시킨다.  
결과적으로 학습이 진행되면서 학습기는 분류하기 어려운 데이터에 집중하게 되고 이전 단계에서 만들어진 학습기는 다음 단계에서 사용할 트레이닝 데이터셋의 가중치를 반영하는데 사용하므로 부스팅은 배깅과는 달리 이전 분류기의 영향을 받는다.

배깅에서는 각 데이터 포인트가 추출될 확률이 모두 동일했지만, 부스팅에서는 각 데이터 포인트에 할당된 가중치에 비례해서 추출된다.

에이다 부스트(Ada Boost)  
약한 학습기 여러 개를 모아 하나의 강한 학습기를 만드는 방법으로 개별적으로는 약한 학습 모델이지만, 이와 같은 모델을 다수 생성하고 부스팅을 적용함으로써 강한 학습기가 만들어진다.

보팅이나 배깅은 모델이 병렬적으로 실행된다. 10개의 모델이 있다면 10개의 모델을 동시에 학습시킬 수 있다는 뜻이다. 이에 반해 부스팅은 여러 약한 학습기가 순차적으로 적용된다. 그 이유는 약한 학습 모델의 학습 이후 판별하지 못한 데이터 포인트에 대해서 가중치를 부여하기 때문이다.

에이다 부스트의 핵심 아이디어는 분류하기 어려운 트레이닝 데이터에 가중치를 더 높이는 것이다. 즉, 이전에 잘못 분류된 트레이닝 데이터 포인트는 가중치가 증가해 오차율이 높아진다. 다음 약한 학습기는 이전 학습기에서 증가한 오차를 낮추는 방향으로 학습하게 된다.  
에이다 부스트는 일반적인 부스팅과는 다르게 약한 학습기를 훈련할 때 훈련 데이터셋 전체를 사용한다. 훈련 샘플은 반복할 때 마다 가중치가 부여되며 이 앙상블은 이전 학습기가 실수한 부분을 학습하는 강력한 분류기를 만든다.

에이다 부스트 알고리즘을 활용해 암 여부를 예측하는 모델을 만든다.

In [5]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_cancer = datasets.load_breast_cancer() # 위스콘신 암 데이터를 불러온다.

In [6]:
# 피쳐, 타겟 데이터 저장
X = raw_cancer.data # 피쳐 데이터를 저장한다.
y = raw_cancer.target # 타겟 데이터를 저장한다.
X.shape

(569, 30)

In [7]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(426, 30) (426,) (143, 30) (143,)


In [8]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [10]:
# 모델 생성 후 데이터 학습
from sklearn.ensemble import AdaBoostClassifier # 에이다 부스트 알고리즘을 사용하기 위해 import 한다.
clf_ada = AdaBoostClassifier() # 에이다 부스트 모델을 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 에이다 부스트 알고리즘을 학습시킨다.
clf_ada.fit(X_tn_std, y_tn)

AdaBoostClassifier()

In [11]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_ada= clf_ada.predict(X_te_std)
print(pred_ada)

[0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


In [12]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_ada)
print(accuracy)

0.9790209790209791


In [13]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_ada, average=None)
print(precision)

[0.96296296 0.98876404]


In [14]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_ada, average=None)
print(recall)

[0.98113208 0.97777778]


In [15]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_ada, average=None)
print(f1)

[0.97196262 0.98324022]


In [16]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_ada)
print(conf_matrix)

[[52  1]
 [ 2 88]]


In [17]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_ada)를 넘겨 분류 리포트를 만든다.
target_names = raw_cancer.target_names
class_report = classification_report(y_te, pred_ada, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

   malignant       0.96      0.98      0.97        53
      benign       0.99      0.98      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

